### 11단계 가변 길이 인수(순전파 편)

- 가변 길이 입출력을 처리할 수 있도록 확장

#### 11.1 Function 클래스 수정

- 가변 길이 입출력을 표현하려면 변수들을 리스트(또는 튜플)에 넣어 처리하면 편함
- Function 클래스는 지금까지처럼 '하나의 인수'만 받고 '하나의 값'만 반환
- 대신 인수와 반환값의 타입을 리스트로 바꾸고, 필요한 변수들을 이 리스트에 넣음

- 기존의 Function 메서드

In [3]:
class Function:
    def __call__(self, input):
        x = input.data # Variable이라는 상자에서 실제 데이터를 꺼냄
        y = self.forward(x) # forward 메서드에서 구체적인 계산
        output = Variable(as_array(y)) # 계산 결과를 Variable에 넣음
        output.set_creator(self) # 자신이 '창조자'라고 원산지 표시
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

- `__call__`메서드의 인수와 반환값을 리스트로 바꿈

In [7]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

#### 11.2 Add 클래스 구현

- 인수와 반환값이 리스트(또는 튜플)여야 한다는것을 주의하고 Add 클래스의 forward 메서드 구현

In [24]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs # 리스트 xs에서 원소 두개를 꺼냄
        y = x0 + x1
        return (y,) # 튜플형태로 반환(return y, 처럼 괄호는)

- Add 클래스 사용

In [15]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


### 12단계 가변 길이 인수(개선 편)

- 첫 번째: Add 클래스를 '사용하는 사람'을 위한 개선
- 두 번째: Add 클래스를 '구현하는 사람'을 위한 개선

#### 12.1 첫 번째 개선: 함수를 사용하기 쉽게

In [19]:
class Function:
    def __call__(self, *inputs): # 별표를 붙이면 리스트를 사용하는 대신 임의 개수의 인수를 건네 함수 호출
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0] # 리스트의 원소가 하나라면 첫 번째 원소를 반환

- 사용 예

In [21]:
def f(*x):
    print(x)

f(1, 2, 3)

(1, 2, 3)


- 이 코드로 알 수 있듯이 함수를 '정의'할 때 인수에 별표를 붙이면 호출할 때 넘긴 인수들을 별표를 붙인 인수 하나로 모아서 받을 수 있다.

In [25]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


#### 12.2 두 번째 개선: 함수를 구현하기 쉽도록

In [29]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # 별표를 붙여 언택
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 추가
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

In [30]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

#### 12.3 add 함수 구현

- Add 클래스를 '파이썬 함수'로 사용할 수 있는 코드를 추가

In [32]:
def add(x0, x1):
    return Add()(x0, x1)

In [33]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


### 13단계 가변 길이 인수(역전파 편)

#### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

- 입력 변수가 여러개인 함수를 다변수 함수라 함
- 다변수 함수에서 하나의 입력 변수에만 주목하여(다른 변수는 상수로 취급) 미분하는 것을 편미분이라 함

In [35]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy): # 입력 1개
        return gy, gy # 출력 2개

- 여러 개의 값을 반환할 수 있게 하려면 Variable 클래스의 backward 메서드를 수정

#### 13.2 Variable 클래스 수정

In [38]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # 출력 변수인 outputs에 담겨 있는 미분값들을 리스트에 담음
            gxs = f.backward(*gys) # 함수 f의 역전파를 호출, f.backward(*gys)처럼 인수에 별표를 붙여 호출하여 리스트를 풀어줌
            if not isinstance(gxs, tuple): # gxs가 튜플이 아니면 튜플로 변환
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장, gxs와 f.inputs의 각 원소는 서로 대응 관계
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

#### 13.3 Square 클래스 구현

In [41]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 수정 전 : x = self.input.data
        gx = 2 * x * gy
        return gx

- add 함수와 square 함수를 사용해보자

In [44]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


### 14단계 같은 변수 반복 사용

- 같은 변수를 반복해서 사용할 셩우 의도대로 동작하지 않을 수 있음, 예) y = add(x, x)

In [46]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 1.0


- 동일한 변수를 사용하여 덧셈을 하면 제대로 미분하지 못함

#### 14.1 문제의 원인

In [48]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx # 1

                if x.creator is not None:
                    funcs.append(x.creator)


- 1 : 출력 쪽에서 전해지는 미분값을 그대로 대입, 같은 변수를 반복해서 사용하면 전파되는 미분값이 덮어 써짐

#### 14.2 해결책

In [60]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None: # 1
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

- 1 : 미분값이 더해지도록 수정

In [61]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0


In [62]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 3.0


#### 14.3 미분값 재설정

In [63]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을)
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0
x.grad 5.0


- 값은 변수를 사용하여 '다른' 계산을 할 경우 계산이 꼬이는 문제 발생

In [66]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self): # 1
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

- 1 : 미분값을 초기화하는 cleargrad 메서드를 추가

In [70]:
# 첫 번째 계산
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

# 두 번째 계산(같은 x를 사용하여 다른 계산을)
x.cleargrad() # 미분값
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0
x.grad 3.0


- 두 번째 backward() 호출 전 cleargrad()호출 하면 변수에 누적된 미분값이 초기화되어 문제 해결

### 15단계 복잡한 계산 그래프(이론 편) 

- 변수와 함수가 꼭 한줄로 연결되리라는 법은 없음
- 같은 변수를 반복해서 사용하거나 여러 변수를 입력받는 함수를 사용하는 계산을 살 후 있음
- 그래프의 '연결된 형태'를 위상(topology)이라 함

#### 15.3 함수 우선순위

- 지금까지는 funcs 리스트에서 '마지막' 원소만 꺼냄
- 리스트에서 적절한 함수를 꺼낼 수 있어야함
- 함수에 '우선순위'를 줄 수 있어야함

### 16단계 복잡한 계산 그래프(구현 편)

#### 16.1 세대 추가

- Variable 클래스와 Function 클래스에 인스턴스 변수 generation을 추가

In [3]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 세대 수를 기록하는 변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록한다(부모세대 + 1)

- Variable 클래스는 generation을 0으로 초기화
- set_creator 메서드가 호출될 때 부보 함수의 세대보다 1만큼 큰 값을 설정

In [5]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) # 1
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

- 1: 입력 변수가 둘 이상이면 가장 큰 generation의 수를 설정

#### 16.3 Variable 클래스의 backward

In [8]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator) # 수정 전 : funcs.append(x.creator)

- add_func 함수 추가
- 리스트에 투가 할 때 funcs.append(f)를 호출했는데, 대신 add_func함수를 호출하도록 변경
- add_func 함수가 리스트를 세대 순으로 정렬하는 역할
- funcs.pop()은 자동으로 세대가 가장 큰 함수를 꺼냄
- add_func 함수를 backward 메서드 안에 중첩 함수로 정의
    - 감싸는 메서드 안에서만 이용
    - 감싸는 메서드에 정의된 변수를 사용

### 17단계 메모리 관리와 순환 참조

- 파이썬 코드를 실행하는 프로그램을 파이썬 인프리터라 함
- 파이썬 인프리터 표준은 CPython
- 메모리 관리 설명은 CPython을 기준으로 함

#### 17.1 메모리 관리

- 파이썬은 필요 없어진 객체를 메모리에서 자동 삭제
- 코드를 제대로 작성하지 않으면 메모리 누수 또는 메모리 부족 문제 발생
- 메모리 관리는 두 가지 방식으로 진행
    - 참조 수를 세는 방식(참조 카운터)
    - 세대를 기준으로 쓸모없어진 객체를 회수하는 방식(가비지 컬렉션)

#### 17.2 참조 카운터 방식의 메모리 관리

- 참조 카운트는 구조가 간단하고 속도가 빠름
- 모든 객체는 참소 카운트가 0인 생태로 생성, 다른 객체가 참조할 때마다 1씩 증가
- 반대로 객체에 대한 참조가 끊길 때마다 1만큼 감소, 0이 되면 파이썬 인프리터가 회수
- 이런 방식으로 객체가 더 이상 필요 없어지면 즉시 메모리 삭제
- 가령 다음과 같은 경우에 참조 카운트가 증가
    - 대입 연산자를 사용할 때
    - 함수에 인수로 전달할 때
    - 컨테이너 타입 객체에 추가할 때

#### 17.3 참조 순환

- 객체가 원 모양을 이루며 서로가 서로를 참조
- 어느 것에도 접근 불가
- 순환 참조의 참조 카운트가 0이 되지 않고, 메모리서 삭제가 안됨
- 그래서 '세대별 가비지 컬렉션'이 등장

#### 17.4 weakref 모듈

- 파이썬에서 weakref.ref 함수를 사용하여 약한 참조를 만들수 있음
- 약한 참조란 다른 객체를 참조하되 참조 카운트는 증가시키지 않는 기능

In [23]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs] # 1
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

- 1: 인스턴스 변수 self.outputs가 대상을 약한 참조로 가리키세 변경

In [24]:
class Variable:
    # ...
    def backward(self):
        # ...
        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
        # ...

- Function 클래스의 outputs를 참조하는 Variable 코드 수정

### 18단계 메모리 절약 모드

- 역전파 시 불필요한 미분 결과를 보관하지 않고 즉시 삭제
- '역전파가 필요 없는 경우용 모드'를 제고해 불필요한 계산 생략

#### 18.1 필요 없는 미분값 삭제

In [26]:
class Variable:
    # ...
    def backward(self, retain_grad=False): # 1
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad: # 2
                for y in f.outputs:
                    y().grad = None  # y is weakref

- 1: 메서드 인수에 retain_grad를 추가, True면 모든 변수가 미분 결과 유지, False면 중간 변수의 미분값을 모두 None으로 설정
- 2: 각 함수의 출렵 변수의 미분값을 유지하지 않도록 y().grad = None으로 설정, y가 약한 참조라 코드가 실해되면 참조 카운트가 0이 되어 미분값 데이터가 메모리에서 삭제

- 신경망에는 학습(training)과 추론(inference)이라는 두 가지 단계가 있음
- 학습 시에는 미분값을 구해야 하지만, 추론 시에는 단순히 순전파만 하기 때문에 중간 계산 결과를 곧바로 버리면 메모리 사용량이 크게 줄일 수 있음

#### 18.3 Config 클래스를 활용한 모드 전환

- '역전파 활성 모드'와 '역전파 비활성 모드'를 전환하는 구조가 필요

In [28]:
class Config:
    enable_backprop = True # True 면 '역전파 활성 모드'

- 설정 데이터는 단 한 군데에만 존재하는게 좋음
- 그래서 Config 클래스는 인스턴스화 하지 않고 '클래스' 상태로 이용
- 인스턴스는 여러 개 생성할 수 있지만 클래스는 항상 하나만 존재

In [ ]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs]) # 1 세대 설정
            for output in outputs:
          for output in outputs:
                output.set_creator(self) # 2 연결 설정
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

- Config.enable_backprop이 True일 때만 역전파 코드가 실행
- 1: '세대'는 역전파 시 노드를 따라가는 순서를 정한느데 사용, '역전파 비활성 모드'에서는 필요하지 않음
- 2: output.set_creator(self)는 계산들의 '연결'을 만드는데, 마찬가지로 '역전파 비활성 모드'에서는 불필요

#### 18.5 with 문을 활용한 모드 전환

- 파이썬에는 후처리를 자동으로 수행하고자 할 때 사용할 수 있는 구문인 with이 있음
- with 문을 사용하는 것으로 'with 블록에 들어갈 때의 처리(전처리)'와 'with 블록을 빠져나올 때의 처리(후처리)'를 자동으로 할 수 있음

In [ ]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

- with using_config('enable_backprop', False): 안에서만 '역전파 비활성 모드'가 됨
- with 블록을 벗어나면 일반모드, 즉 '역전파 활성 모드'로 돌아감

- with 문을 사용한 모든 전환 구현은 contextlib 모듈을 사용하면 가장 쉽게 구현할 수 있음

In [38]:
import contextlib

@contextlib.contextmanager
def config_test():
    print('start') # 전처리
    try:
        yield
    finally:
        print('done') # 후처리

with config_test():
    print('process...')

start
process...
done


- @contextlib.contextmanager 데코레이터를 달면 문맥을 판단한느 함수가 만들어짐
- 이 함수 안에서 yield 전에는 전처리 로직을, yield 다음에는 후처리 로직을 작성

- using_config 함수 구현

In [40]:
import contextlib

class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

- name은 타입이 str, 사용할 Config 속성의 이름을 가리킴
- name을 getattr 함수에 넘겨 Config 클래스에서 꺼내온 다음 setattr 함수를 사용하여 새로운 값을 설정
- with 블록에 들어갈 때 name으로 지정한 Config 클래스 속성이 value로 설정
- with 블록을 빠져나오면서 원래값(old_value)으로 복원

In [46]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

- 역전파가 필요 없는 경우 with 블록에서 순전파 코드만 실행
- 불필요한 계산을 생략하고 메모리 절약

In [48]:
def no_grad():
    return using_config('enable_backprop', False)

with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)

- no_grad 함수는 단순히 using_config('enable_backprop', False)를 호출하는 코드를 return, 편의 함수

### 19단계 변수 사용성 개선

- DeZero를 더 쉽게 사용하도록 개선

#### 19.1 변수 이름 지정

- 많은 변수를 처리하기 때문에 변수들을 서로 구분할 필요가 있음
- Variable 클래스에 name이라는 인스턴스 변수를 추가해 변수에 '이름'을 붙여줄 수 있도록 설정

In [50]:
class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

- 변수에 이름을 붙일 수 있다면, 계산 그래프를 시각화할 때 변수 이름을 그래프에 표시할 수 있음

#### 19.2 ndarray 인스턴스 함수

- Variable 클래스는 스칼라는 무시하고 ndarray만 취급하기로 함
- Variable 인스턴스를 ndarray 인스턴스 처럼 보이게 함

In [52]:
import numpy as np
x = np.array([[1, 2, 3],[4, 5, 6]])
x.shape

(2, 3)

- shape는 다차원 배열의 형상을 알려줌
- Variable 인스턴스에서도 할 수 있도록 확장

In [54]:
class Variable:
    # ...
    @property
    def shape(self):
        return self.data.shape

- shape라는 메서드를 추가한 후 실제 데이터의 shape를 반환하도록 함
- @property 덕분에 shape 메서드를 인스턴스 변수처럼 사용할 수 있게 됨

In [58]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x.shape)

(2, 3)


- ndarray의 다른 인스턴스 변수들을 추가

In [ ]:
class Variable:
    # ... 
    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

- ndim: 차원 수, size: 원소 수, dtype: 데이터 타입

#### 19.3 len 함수와 print 함수

In [ ]:
class Variable:
    # ... 
    def __len__(self):
        return len(self.data)

- `__len__`이라는 특수 메서드를 구현하면 Variable 인스턴스에 대해서도 len 함수를 사용할 수 있음

In [ ]:
class Variable:
    # ... 
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

- print 함수가 출력해주는 문자열을 입맛에 맞게 정의하려면 `__repr__`메서드를 재정의하면 됨

### 20단계 연산자 오버로드(1)

- `+`와 `*`연산자를 지원하도록 확장

#### 20.1 Mul 클래스 구현

In [64]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

- 파이썬 함수로 사용할 수 있도록 함

In [66]:
def mul(x0, x1):
    return Mul()(x0, x1)

#### 20.2 연산자 오버로드

- `__mul__`메서드를 정의(구현)하면 * 연산자를 사용할 때 `__mul__`메서드가 호출 됨

In [ ]:
class Variable:
    # ... 
    
Variable.__add__ = add
Variable.__mul__ = mul

### 21단계 연산자 오버로드(2)

- Variable  인스턴스와 ndarray 인스턴스, int와 float 등도 함께 사용할 수 있도록 확장

#### 21.1 ndarray와 함께 사용하기

In [69]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

- 자동으로 Variable 인스턴스로 변환해주는 편의 함수
- 인수 obj가 Variable 인스턴스 또는 ndarray 인스턴스라고 가정
- obj가 Variable 인스턴스면 그대로 반환, 아니면 Variable 인스턴스로 변환 후 반환

In [72]:
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)

- inputs에 담긴 각각의 원소 x를 Variable 인스턴스로 변환
- ndarray 인스턴스가 주어지면 Variable 인스턴스로 변환

#### 21.2 float, int와 함께 사용하기

In [ ]:
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

- as_array 함수를 사용해 float나 int인 경우 ndarray 인스턴스로 변환
- ndarray 인스턴스는 Function 클래스에서 Variable인스턴스로 변환

#### 21.3 문제점 1: 첫 번째 인수가 float나 int인 경우

- 이항 연산자의 경우 피연산자(항)의 위치에 따라 호출되는 특수 메서드가 다름
- 곱셍의 경우 피연산자가 좌항이면 `__mul__`메서드 호출, 우향이면 `__rmul__`메서드 호출해 해결

In [75]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

#### 21.4 문제점 2: 좌항이 ndarray 인스턴스인 경우

- 좌항이 ndarray 인스턴스고 우항이 Variable 인스턴스인 경우, 좌항인 ndarray 인스턴스의 `__add__`메서드가 호출 됨
- 우항인 Variable 인스턴스의 `__radd__`메서드가 호출되려면 '연산자 우선수위'를 지정해야 함

In [ ]:
class Variable:
ass Variable:
    __array_priority__ = 200 # Variable 인스턴스의 연산자 우선순위를 ndarray 인스턴스의 연산자 우선수위 보다 높임
    # ...

- 좌항이 ndarray 인스턴스라 해도 우항인 Variable 인스턴스의 연산자 메서드가 우선적으로 호출

### 22단계 연산자 오버로드(3)

- 새로운 연산자를 추가하는 순서
    - Function 클래스를 상속하여 원하는 함수 클래스를 구현
    - 파이썬 함수로 사용할 수 있도록 함
    - Variable 클래스의 연산자를 오버로드 함

#### 22.1 음수(부호 변환)

- 역전파는 상류(출력 쪽)에서 전해지는 미분에 -1을 곱하여 하류로 흘려보내 주면 됨

In [79]:
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

#### 22.2 뺄셈

- 역전파는 상류에서 전해지는 미분값에 1을 곱한 값이 x0의 미분 결과가 되며, -1을 곱한 값이 x1의 미분 결과가 됨

In [81]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

- 좌항이 Variable 인스턴스가 아닌 경우 `__rsub__`메서드가 호출 되어야 함

In [83]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)

Variable.__rsub__ = rsub

#### 22.3 나눗셈

In [88]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0) # x0와 x1의 순서를 바꾼다.

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

#### 22.4 거듭제곱

- 이 책에서는 밑이 x인 경우만 미분

In [90]:
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow

- Pow 클래스를 초기화할 때 지수 c를 제공
- 순전파 메서드인 forward(x)는 밑에 해당하는 x만(즉, 하나의 항만) 받게 함

### 23단계 패키지로 정리

### 24단계 복잡한 함수의 미분

#### 24.1 Sphere 함수

- 단순희 두 개의 입력 변수를 제곱하여 더하는 변수

In [2]:
import numpy as np
from dezero import Variable

def sphere(x, y):
    z = x ** 2 + y ** 2
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)
z.backward()
print(x.grad, y.grad)

2.0 2.0


#### 24.2 matyas 함수

In [5]:
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)
z.backward()
print(x.grad, y.grad)

0.040000000000000036 0.040000000000000036


#### 24.3 Goldstein-Price 함수

In [7]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0
